In [20]:
import requests
import bs4
import datetime
import pandas as pd

In [46]:
url = 'http://app.bde.es/sew_www/GestorDePeticiones?IdOperacion=besew_www_DescargaUnidadesSEC2010UltimaFechaOP'

In [ ]:
payload = {"ComboBox": "", 
           "form:BotoneraUsuarioLimitado_3": "CSV Downloads", 
           "token": token, 
           "form_SUBMIT": form_submit, 
           "javax.faces.ViewState": javax}
r = requests.post(url)

In [21]:
r = requests.get(url=url)
soup = bs4.BeautifulSoup(r.content)

In [23]:
token = soup.find("input", attrs={"name": "token", "type": "hidden"})["value"]
form_submit = soup.find("input", attrs={"name": "form_SUBMIT", "type": "hidden"})["value"]
javax = soup.find("input", attrs={"name": "javax.faces.ViewState", "type": "hidden"})["value"]

In [48]:
with requests.Session() as s:
    
    # First request, retrieving hidden values.
    r = s.get(url=url)
    soup = bs4.BeautifulSoup(r.content)
    
    token = soup.find("input", attrs={"name": "token", "type": "hidden"})["value"]
    form_submit = soup.find("input", attrs={"name": "form_SUBMIT", "type": "hidden"})["value"]
    javax = soup.find("input", attrs={"name": "javax.faces.ViewState", "type": "hidden"})["value"]
    
    # Second request, sending payload as a post.
    
    payload = {"ComhtmlboBox": "", 
               "form:BotoneraUsuarioLimitado_3": "CSV Downloads", 
               "token": token, 
               "form_SUBMIT": form_submit, 
               "javax.faces.ViewState": javax}
    
    form_url = "http://app.bde.es/sew_www/faces/sew_wwwias/jsp/op/DescargaUnidadesSEC2010UltimaFechaOP/DescargaUnidadesSEC2010UltimaFecha.jsp"
    r = s.post(form_url, data=payload)
    soup = bs4.BeautifulSoup(r.content)
    
    # Third request, downloading lastest available data. 
    raw_url = 'http://app.bde.es'
    csv_link = soup.find("a", attrs={"id": "form:Enlace_Link"})["href"]
    csv_file = s.get(raw_url + csv_link)
    
with open("Unidades.csv", "wb") as f:
    f.write(csv_file.content)

18384942

In [53]:
df = pd.read_csv("Unidades.csv", skiprows=3, encoding ='iso-8859-1')

In [23]:
df.head()

,NIF,LEI,NOMBRE,FECHA DESDE,FECHA HASTA,SECTOR
0,A01001833,NaN,ACENOR SA,02/04/2020,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...
1,A06005599,NaN,MERCADOS CENTRALES DE ABASTECIMIENTO DE BADAJO...,06/03/2020,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...
2,A10037943,NaN,DISTRIBUCION CLIMATIZADA SA,06/03/2020,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...
3,A11002565,NaN,"COMPAÑIA INMOBILIARIA DARSA GADITANA, SA",02/04/2020,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...
4,A11484490,NaN,RED IBEROAMERICANA DE LOGISTICA Y COMERCIO SA,01/01/2008,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...


In [17]:
class Scraper(object):
    """
    Scraper.
    
    Parameters:
        url: String. BDE url.
    """
    
    def __init__(self, url):
        """
        Init method.
        """
        
        self.url = url
        self.fetch_time = None
        
        
    def __repr__(self):
        """
        Visualization
        """
        return self.url + "\n" + self.fetch_time
    
    
    def fetching_data(self, filename: str):
        """
        Retrieve data and save it as a csv file.
        
        Parameters:
            filename: String. Filename path. E.g.: "Unidades.csv"
        
        Returns:
            Fetched data saved in a CSV file.
        """
        
        with requests.Session() as s:
    
            # First request, retrieving hidden values.
            r = s.get(url=url)
            soup = bs4.BeautifulSoup(r.content)

            token = soup.find("input", attrs={"name": "token", "type": "hidden"})["value"]
            form_submit = soup.find("input", attrs={"name": "form_SUBMIT", "type": "hidden"})["value"]
            javax = soup.find("input", attrs={"name": "javax.faces.ViewState", "type": "hidden"})["value"]

            # Second request, sending payload as a post.
            payload = {"ComboBox": "", "form:BotoneraUsuarioLimitado_3": "CSV Downloads", 
                       "token": token, "form_SUBMIT": form_submit, "javax.faces.ViewState": javax}

            form_url = "http://app.bde.es/sew_www/faces/sew_wwwias/jsp/op/DescargaUnidadesSEC2010UltimaFechaOP/DescargaUnidadesSEC2010UltimaFecha.jsp"
            r = s.post(form_url, data=payload)
            soup = bs4.BeautifulSoup(r.content)

            # Third request, downloading lastest available data. 
            raw_url = 'http://app.bde.es'
            csv_link = soup.find("a", attrs={"id": "form:Enlace_Link"})["href"]
            csv_file = s.get(raw_url + csv_link)

        with open(filename, "wb") as f:
            f.write(csv_file.content)
            
        self.fetch_time = datetime.datetime.now().strftime('%d/%m/%Y %H:%M')
            
            
    def reading_csv(self, filename: str):
        """
        Read CSV file into a data frame.
        
        Parameters: 
            filename: String. CSV file to read.
        
        Returns:
            Read file into a dataframe.
        """
        
        df = pd.read_csv(filename, skiprows=3, encoding ='iso-8859-1')
        return df
        

In [21]:
url = 'http://app.bde.es/sew_www/GestorDePeticiones?IdOperacion=besew_www_DescargaUnidadesSEC2010UltimaFechaOP'
bde = Scraper(url)
bde.fetching_data("Unidades_test.csv")
df = bde.reading_csv("Unidades_test.csv")
df

,NIF,LEI,NOMBRE,FECHA DESDE,FECHA HASTA,SECTOR
0,A01001833,NaN,ACENOR SA,02/04/2020,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...
1,A06005599,NaN,MERCADOS CENTRALES DE ABASTECIMIENTO DE BADAJO...,06/03/2020,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...
2,A10037943,NaN,DISTRIBUCION CLIMATIZADA SA,06/03/2020,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...
3,A11002565,NaN,"COMPAÑIA INMOBILIARIA DARSA GADITANA, SA",02/04/2020,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...
4,A11484490,NaN,RED IBEROAMERICANA DE LOGISTICA Y COMERCIO SA,01/01/2008,--/--/----,1.1.1.1-Sociedades no financieras públicas dep...
...,...,...,...,...,...,...
115919,V99182503,NaN,ENTIDAD URBANISTICA DE CONSERVACION DEL PARQUE...,29/06/2007,--/--/----,1.5-Instituciones sin fines de lucro al servic...
115920,V99185308,NaN,CLUB DEPORTIVO ELEMENTAL JESUS MARIA-EL SALVADOR,28/06/2007,--/--/----,1.5-Instituciones sin fines de lucro al servic...
115921,V99303570,NaN,COMPROMISO CON ARAGON,26/01/2011,--/--/----,1.5-Instituciones sin fines de lucro al servic...
115922,V99328619,NaN,GRUPO MUNICIPAL SOCIALISTA DEL AYUNTAMIENTO DE...,13/06/2011,--/--/----,1.5-Instituciones sin fines de lucro al servic...


In [19]:
bde

http://app.bde.es/sew_www/GestorDePeticiones?IdOperacion=besew_www_DescargaUnidadesSEC2010UltimaFechaOP
14/06/2021 17:27